In [1]:
import numpy
import pandas as pd
from ctypes import *
import math
from pandas.testing import assert_frame_equal

In [3]:
so_file = "C:\\Users\\maryl\\Documents\\my_projects\\bezpieczenstwo_projekt\\encoder.so"
encoder = CDLL(so_file)
print(type(encoder))

<class 'ctypes.CDLL'>


In [4]:
p = 0
q = 0
num = 1024
#alghoritm won't work if n is lesser than the message
p = encoder.get_prime()
q = encoder.get_prime()
n = (p*q)
phi = (p-1)*(q-1)
e = encoder.relative_prime(phi)
d = encoder.get_d_parameter( e,  phi)
Enum = (encoder.mod_exp(num, e, n))
Denum = (encoder.mod_exp(Enum, d, n))
assert num == Denum, 'decrypted message should be the same as the original message'

In [5]:
df = pd.read_csv("Crop_Recommendation.csv")
df.head()

,Nitrogen,Phosphorus,Potassium,Temperature,Humidity,pH_Value,Rainfall,Crop
0,90,42,43,20.879744,82.002744,6.502985,202.935536,Rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,Rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,Rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,Rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,Rice


In [6]:
df['Crop'].unique()
dic = dict(zip(df['Crop'].unique(),range(22)))

In [7]:
#df = df.drop('Crop', axis =1)
columns_to_modify = ['Temperature', 'Humidity', 'pH_Value', 'Rainfall']
new_columns = ['Temperature_dec', 'Humidity_dec', 'pH_Value_dec', 'Rainfall_dec']

idx = 0
for column in columns_to_modify:
    df[new_columns[idx]] = df[column]%1
    idx+=1

for column in new_columns:
    df[column] = df[column]*100

df = df.replace({'Crop': dic})


df[columns_to_modify] = df[columns_to_modify].applymap(math.floor)
df[new_columns] = df[new_columns].applymap(math.floor)
df.info()
df.describe()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Nitrogen         2200 non-null   int64
 1   Phosphorus       2200 non-null   int64
 2   Potassium        2200 non-null   int64
 3   Temperature      2200 non-null   int64
 4   Humidity         2200 non-null   int64
 5   pH_Value         2200 non-null   int64
 6   Rainfall         2200 non-null   int64
 7   Crop             2200 non-null   int64
 8   Temperature_dec  2200 non-null   int64
 9   Humidity_dec     2200 non-null   int64
 10  pH_Value_dec     2200 non-null   int64
 11  Rainfall_dec     2200 non-null   int64
dtypes: int64(12)
memory usage: 206.4 KB


,Nitrogen,Phosphorus,Potassium,Temperature,Humidity,pH_Value,Rainfall,Crop,Temperature_dec,Humidity_dec,pH_Value_dec,Rainfall_dec
0,90,42,43,20,82,6,202,0,87,0,50,93
1,85,58,41,21,80,7,226,0,77,31,3,65
2,60,55,44,23,82,7,263,0,0,32,84,96
3,74,35,40,26,80,6,242,0,49,15,98,86
4,78,42,42,20,81,7,262,0,13,60,62,71


In [8]:
og_df = df.copy()

for column in df.columns:
    df[column] = df[column].apply(lambda x: encoder.mod_exp(c_longlong(x), e, n))

print(df == og_df)

      Nitrogen  Phosphorus  Potassium  Temperature  Humidity  pH_Value  \
0        False       False      False        False     False     False   
1        False       False      False        False     False     False   
2        False       False      False        False     False     False   
3        False       False      False        False     False     False   
4        False       False      False        False     False     False   
...        ...         ...        ...          ...       ...       ...   
2195     False       False      False        False     False     False   
2196     False       False      False        False     False     False   
2197     False       False      False        False     False     False   
2198     False       False      False        False     False     False   
2199     False       False      False        False     False     False   

      Rainfall   Crop  Temperature_dec  Humidity_dec  pH_Value_dec  \
0        False   True            False   

In [9]:
print(assert_frame_equal(df, og_df))

AssertionError: DataFrame.iloc[:, 0] (column name="Nitrogen") are different

DataFrame.iloc[:, 0] (column name="Nitrogen") values are different (97.86364 %)
[index]: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]
[left]:  [111144, 34026, 101392, 112482, 43198, 31782, 31782, 154196, 35249, 142569, 86915, 111144, 43198, 8903, 154196, 101392, 34026, 86915, 66629, 79808, 35249, 43322, 6741, 98359, 146963, 103466, 7021, 7021, 101392, 72530, 37445, 34515, 34026, 146963, 79808, 95202, 120722, 95202, 101392, 74629, 133872, 1393, 98359, 114284, 34026, 86915, 43322, 112482, 68455, 79808, 101392, 43322, 8903, 109105, 95202, 100427, 112482, 86915, 148191,

In [10]:
df.head()

,Nitrogen,Phosphorus,Potassium,Temperature,Humidity,pH_Value,Rainfall,Crop,Temperature_dec,Humidity_dec,pH_Value_dec,Rainfall_dec
0,111144,26055,69952,8779,114284,140922,22253,0,78641,0,7428,8903
1,34026,153003,22752,81152,135379,4179,92976,0,66629,37685,94151,109105
2,101392,13015,30530,92811,114284,4179,128098,0,0,82201,72530,142009
3,112482,17304,15435,38347,135379,140922,98906,0,46920,37393,146963,113908
4,43198,26055,26055,8779,57867,4179,63794,0,33776,101392,133872,148191


In [11]:
def decrypt():
    df[column] = df[column].apply(lambda x: encoder.mod_exp(c_longlong(x), d, n))

In [13]:
for column in df.columns:
    df[column] = df[column].apply(lambda x: encoder.mod_exp(c_longlong(x), d, n))

print(og_df == df)

      Nitrogen  Phosphorus  Potassium  Temperature  Humidity  pH_Value  \
0         True        True       True         True      True      True   
1         True        True       True         True      True      True   
2         True        True       True         True      True      True   
3         True        True       True         True      True      True   
4         True        True       True         True      True      True   
...        ...         ...        ...          ...       ...       ...   
2195      True        True       True         True      True      True   
2196      True        True       True         True      True      True   
2197      True        True       True         True      True      True   
2198      True        True       True         True      True      True   
2199      True        True       True         True      True      True   

      Rainfall  Crop  Temperature_dec  Humidity_dec  pH_Value_dec  \
0         True  True             True     

In [14]:
print(assert_frame_equal(df, og_df))

None


In [15]:
df.head()

,Nitrogen,Phosphorus,Potassium,Temperature,Humidity,pH_Value,Rainfall,Crop,Temperature_dec,Humidity_dec,pH_Value_dec,Rainfall_dec
0,90,42,43,20,82,6,202,0,87,0,50,93
1,85,58,41,21,80,7,226,0,77,31,3,65
2,60,55,44,23,82,7,263,0,0,32,84,96
3,74,35,40,26,80,6,242,0,49,15,98,86
4,78,42,42,20,81,7,262,0,13,60,62,71


In [16]:
for column in new_columns:
    df[column] = df[column]/100

df

idx = 0
for column in columns_to_modify:
    df[column] += df[new_columns[idx]]
    idx+=1

for column in new_columns:
    df = df.drop([column], axis = 1)

In [17]:
dic = dict(zip(dic.values(),dic.keys()))
df = df.replace({'Crop': dic})

In [18]:
df

,Nitrogen,Phosphorus,Potassium,Temperature,Humidity,pH_Value,Rainfall,Crop
0,90,42,43,20.87,82.00,6.50,202.93,Rice
1,85,58,41,21.77,80.31,7.03,226.65,Rice
2,60,55,44,23.00,82.32,7.84,263.96,Rice
3,74,35,40,26.49,80.15,6.98,242.86,Rice
4,78,42,42,20.13,81.60,7.62,262.71,Rice
...,...,...,...,...,...,...,...,...
2195,107,34,32,26.77,66.41,6.78,177.77,Coffee
2196,99,15,27,27.41,56.63,6.08,127.92,Coffee
2197,118,33,30,24.13,67.22,6.36,173.32,Coffee
2198,117,32,34,26.27,52.12,6.75,127.17,Coffee
